<a href="https://colab.research.google.com/github/sh-Gabriel/Atrial-Enlargement-Detection/blob/main/TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports

from google.colab import drive

import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

from PIL import Image
import numpy  as np
import pandas as pd
import cv2    as cv
np.random.seed(42)
import random
import os

import glob
import csv
from itertools import islice
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import tensorflow as tf
from tensorflow import keras

from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.pipeline import Pipeline
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from sklearn import metrics
from tensorflow import keras
from keras import layers 

!pip3 install mxnet
#augmentation
import matplotlib.pyplot as plt
import mxnet as mx
from mxnet.gluon.data.vision import transforms
#cv2 importado acima



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#mount and defines

# drive.mount('/content/drive/MyDrive/TCC/',force_remount=True)
drive.mount('/content/drive',force_remount=True)
PATH             = 'drive/MyDrive/TCC/Dataset_V1/'
norm             = "N"
doente           = "AAE_EP"
left             = "Torax_LAT_E"
right            = "Torax_LAT_D"
stop_n = stop_s  = 6 # voltei pra 6 pra fazer os testes
current_s        = 6
# Imagem é mto grande para ser alocada na gpu, não dá pra usar esses valores
# shape_w, shape_h = 2928, 2328
shape_w, shape_h = 224,224
EPOCHS           = 12
# !git clone https://github.com/pjreddie/darknet.git
# !cd drive/MyDrive/TCC/darknet; make
!nvidia-smi --query-gpu=memory.total --format=csv
!nvidia-smi

Mounted at /content/drive
memory.total [MiB]
15109 MiB
Sat Nov 12 17:27:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                 

Coloquei os defs e a chamada do join_open_files tudo junto, porque basicamente eles fazem uma coisa só que é o join_open_files e assim, não precisa ficar rodando os defs de novo toda vez que muda alguma coisa

In [5]:
from google.colab.patches import cv2_imshow
from numpy import expand_dims
from tensorflow.keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot
from io import BytesIO
from PIL import Image
from tensorflow.image import adjust_contrast
from tensorflow import numpy_function
def rotation(img):
  dim = img.shape[:2]
  center = (dim[0]/2, dim[1]/2)
  rot = cv.getRotationMatrix2D(center, 15, 1)
  rotated = cv.warpAffine(img, rot,dim)
  return rotated

def translation(img):
  T = np.float32([[1, 0, 0], [0, 1, -50]])
  translated = cv.warpAffine(img, T, img.shape[:2])
  return translated

def enbrighten(img):
  sam = expand_dims(img, 0)

  imageDataGenerator_obj = ImageDataGenerator(brightness_range=[0.9,0.9])

  iterator = imageDataGenerator_obj.flow(sam, batch_size=1)

  result = iterator.next()[0].astype('uint8')
  
  return result

def change_contrast(img):
  contrast = tf.keras.layers.RandomContrast(0.3)

  cv2_imshow(contrast(img).numpy().astype('uint8'))

  imageDataGenerator_obj = ImageDataGenerator(brightness_range=[0.2,1.0])

  return contrast(img).numpy().astype('uint8')

In [6]:
#1 chamada = 1 load em 1 classe com 1 lateralidade
# retorno do vetor com os nomes das imagens
def openImgs(classe,lateralidade):
  r_v = [] 
  for subdir, dirs, files in os.walk(PATH+classe):
    if (files and lateralidade in subdir):
      r_v += [subdir+'/'+files[0]]
  return shuffle(r_v, random_state = 42)

#Retorna vetores para entrada de modelo
#  X valores e Y labels
#  Y label da imagem (1 = presença de doença)

#augmentation
def augmentation(X, Y):
  to_augment = X[stop_n:]
  for img in to_augment:
    cv2_imshow(img)
    # break
    # #rotate (15º max)
    # rot = cv.getRotationMatrix2D(center, 15, 1)
    # rotation = cv.warpAffine(img, rot,dim)
    # cv2_imshow(rotation)
    X += [rotation(img)]
    # #translation (+-50 OK)
    # T = np.float32([[1, 0, 0], [0, 1, -50]])
    # translation = cv.warpAffine(img, T, dim)
    # cv2_imshow(translation)
    X += [translation(img)]
    # X +=
    # #brightness (+-10?)
    # X +=
    X += [enbrighten(img)]
    # print(transforms.RandomBrightness(10))
    # #contrast (+-1 a 4 de intensidade?)
    X += [change_contrast(img)]
    # #para cada aumento + 1 label pro vetor y
    Y += [1] * 4

def join(lateralidade):
  #print("Accessing files...")
  X=[]
  Y=[]
  for f in islice(openImgs(norm, lateralidade), 0, stop_n):
      X += [f]
      Y += [0]

  for f in islice(openImgs(doente, lateralidade), 0, current_s):
      X += [f]
      # AINDA NÃO FIZ OS TESTES
      # X += [rotation(f)]
      # X += [translation(f)]
      # X += [enbrighten(f)]
      # X += [change_contrast(f)]
      Y += [1]
  return X,Y

def join_open_files(lateralidade=left, reshape=True):
  X, y = join(lateralidade)
  Xl = []
  #coloca no vetor imagens abertas
  for i in X:
    # cv2_imshow(cv.imread(i, 0))
    if reshape:
      Xl.append(cv.resize(cv.imread(i), (shape_w, shape_h)))
    else:
      Xl.append(cv.imread(i))

  #ta pegando a imagem de 224, mudar a chamada pra aplicar o augmentation na imagem original
  # if(current_s != stop_s):
  #   print("Antes da chamada")
  #   augmentation(Xl,y);
  return np.array(Xl), np.array(y)

X, y = join_open_files() #Coloca as imagens já abertas num vetor para classificá-las e suas respectivas classificações em outro

In [7]:
X.shape


(12, 224, 224, 3)

In [8]:
def custom_VGG16():
  vgg16 = keras.applications.VGG16(input_shape=(shape_h, shape_w, 3), include_top=False, weights="imagenet", classes=2);

  last_layer = vgg16.get_layer('block5_pool')
  last_output = last_layer.output
  extraLayer = tf.keras.layers.GlobalMaxPooling2D()(last_output)
  extraLayer = tf.keras.layers.Dense(512, activation='relu')(extraLayer)
  extraLayer = tf.keras.layers.Dropout(0.5)(extraLayer)
  extraLayer = tf.keras.layers.Dense(2, activation='sigmoid')(extraLayer)
  
  vgg16 = keras.Model(vgg16.input, extraLayer)
  
  return vgg16
  

In [ ]:
def custom_InceptionV3():
  iv3 = keras.applications.InceptionV3(input_shape=(shape_w, shape_h, 3), include_top=False, weights="imagenet");

  last_layer  = iv3.get_layer('mixed10')
  last_output = last_layer.output
  extraLayer = tf.keras.layers.GlobalMaxPooling2D()(last_output)
  extraLayer = tf.keras.layers.Dense(512, activation='relu')(extraLayer)
  extraLayer = tf.keras.layers.Dropout(0.5)(extraLayer)
  extraLayer = tf.keras.layers.Dense(2, activation='sigmoid')(extraLayer)

  model = keras.Model(iv3.input, extraLayer)

  return model
    
    

In [ ]:
def custom_ResNet50V2():
  resnet50v2 = keras.applications.ResNet50V2(input_shape=(shape_w, shape_h, 3), include_top=False, weights="imagenet");
  
  last_layer  = resnet50v2.get_layer('post_relu')
  last_output = last_layer.output
  extraLayer = tf.keras.layers.GlobalMaxPooling2D()(last_output)
  extraLayer = tf.keras.layers.Dense(512, activation='relu')(extraLayer)
  extraLayer = tf.keras.layers.Dropout(0.5)(extraLayer)
  extraLayer = tf.keras.layers.Dense(2, activation='sigmoid')(extraLayer)

  model = keras.Model(resnet50v2.input, extraLayer)

  return resnet50v2

#A principio tá rodando sem precisar colocar as layers no final

In [12]:
def displayHistory(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric])
  plt.title('model '+metric)
  plt.ylabel(metric)
  plt.xlabel('epoch')
  plt.legend(['train', 'test'], loc='upper left')
  plt.show()

In [15]:

from sklearn import svm
#Leave one patient out using SVC from above
import numpy as np
from keras.callbacks import EarlyStopping,ModelCheckpoint
from sklearn.model_selection import LeaveOneOut

def custom_LOO(X, y, model):
  fold_var = 1
  loo = LeaveOneOut()
  loo.get_n_splits(X)
  results = []
  nsamples, nx, ny, channels = X.shape

  checkpoint_path = 'drive/MyDrive/TCC/checkpoints/'
  checkpoint_name = 'pesos_{epoch:02d}.hdf5'
  
    # acabo de comentar
  # model.save_weights('drive/MyDrive/TCC/checkpoints/cp.ckpt')
  
  #loss resolvi o (None, ) (None, x, y,)
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
  
def custom_LOO(X, y, model, name):
  fold_var = 1
  loo = LeaveOneOut()
  loo.get_n_splits(X)
  results = []
  nsamples, nx, ny, channels = X.shape

  checkpoint_path = 'drive/MyDrive/TCC/checkpoints/'
  checkpoint_name = name+'pesos_{epoch:02d}.hdf5'
  
    # acabo de comentar
  # model.save_weights('drive/MyDrive/TCC/checkpoints/cp.ckpt')
  
  #loss resolvi o (None, ) (None, x, y,)
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, optimizer='adam', metrics=['accuracy'])
  
  h = []

  for train_index, test_index in loo.split(X):
                      #----- Mudei os vermose pra 0 no ModelCheckpoint e no fit pra não aparecer os prints -----#
    checkpoint = ModelCheckpoint(filepath=checkpoint_path+checkpoint_name, monitor='val_accuracy', verbose = 0, mode='max', save_weights_only=True)

    callback_list = [checkpoint]

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
 
    history = model.fit(X_train, y_train, epochs=EPOCHS, callbacks=checkpoint, validation_split = 0.1, verbose = 0)
    # transfer_model.fit(train1_data, train1_labels, batch_size = 15, epochs=5, validation_data=(test1_data, test1_labels) callbacks=[lr_reduce,checkpoint])

    model.load_weights(checkpoint_path+name+'pesos_'f'{fold_var:02}.hdf5')
    
    # print("--- [==============================] Evaluate")
    results += [model.evaluate(X_test, y_test)[1]]
    # print(results)
    # results = dict(zip(model.metrics_names, results))
    tf.keras.backend.clear_session()
    # break;
    fold_var += 1
    h += history.history['accuracy']
    displayHistory(history, 'accuracy')

  return results

In [ ]:
print(np.sum(custom_LOO(X, y, custom_VGG16(), "VGG16"))/12)

In [ ]:
print(np.sum(custom_LOO(X, y, custom_InceptionV3(), "Incep"))/12)
#custom_InceptionV3().summary()

Epoch 1/12
1/1 [==============================] - ETA: 0s - loss: 0.1149 - accuracy: 1.0000
Epoch 1: saving model to drive/MyDrive/TCC/checkpoints/Inceppesos_01.hdf5
1/1 [==============================] - 9s 9s/step - loss: 0.1149 - accuracy: 1.0000 - val_loss: 5.3644e-07 - val_accuracy: 1.0000
Epoch 2/12
1/1 [==============================] - ETA: 0s - loss: 2.3278 - accuracy: 0.7778
Epoch 2: saving model to drive/MyDrive/TCC/checkpoints/Inceppesos_02.hdf5
1/1 [==============================] - 1s 1s/step - loss: 2.3278 - accuracy: 0.7778 - val_loss: 45.8353 - val_accuracy: 0.0000e+00
Epoch 3/12
1/1 [==============================] - ETA: 0s - loss: 5.2392 - accuracy: 0.6667
Epoch 3: saving model to drive/MyDrive/TCC/checkpoints/Inceppesos_03.hdf5
1/1 [==============================] - 1s 1s/step - loss: 5.2392 - accuracy: 0.6667 - val_loss: 52.9803 - val_accuracy: 0.0000e+00
Epoch 4/12
1/1 [==============================] - ETA: 0s - loss: 1.4120 - accuracy: 0.8889
Epoch 4: saving mo

In [ ]:
#print(np.sum(custom_LOO(X, y, custom_ResNet50V2(), "ResNet"))/12)
print(custom_ResNet50V2().summary())

Model: "resnet50v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_15[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                         

Verificar pra fazer algo customizado e alterar o classiicador por meio de um laço, fica bala

Antes tava fazendo 50% treino e 50% teste, por isso o 0.333333...

> Mount e Defines

>  Funções que abrem a imagem

> *   openImgs()
*   join()
*   joi_open_file()
*   def VGG16(), InceptionV3() e ResNet50V2()
*   chamadas das redes usando o Leave-One-Out
*   chamada dos summary() das redes

> Problemas não resolvidos
>* por algum caralho de motivo, a rede recebe os parametros de shape_w e shape_h invertidos. Se tenta inverter a rede dá um erro de "ResourceExhaustedError: Graph execution error"

* por outro caralho de motivo, conforme vai rodando, vão sendo criados mais checkpoints que filho de pobre esperando bolsa familia 

> Problemas resolvidos

In [ ]:
custom_VGG16().summary()
custom_ResNet50V2().summary()
#custom_InceptionV3().summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
session = InteractiveSession(config=config)

# model code as usual down here